<p><font size=18>Lesson 10: Recommender Systems 1</font></p>

In [1]:
# execute to import notebook styling for tables and width etc.
from IPython.core.display import HTML
import urllib.request
# response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
# HTML(response.read().decode("utf-8"));

In [2]:
import pandas as pd

# Practice

Follow the examples and use the code files provided from chapters 1-4 in **Hands-On Recommendation Systems with Python** by Rounak Banik to do the following self-assessment exercises.  Learn by doing!

## Simple Recommender

### <font color = "blue"> Self-Assessment: Load and Display </font>

Load the data set **ted_main.csv** and display the first 5 rows. This data set can be found in the presentation download for this lesson.  More information about this data set <a href = https://www.kaggle.com/rounakbanik/ted-talks> here </a>.  

In [3]:
df = pd.read_csv("data/ted-talks/ted_main.csv")
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550 entries, 0 to 2549
Data columns (total 17 columns):
comments              2550 non-null int64
description           2550 non-null object
duration              2550 non-null int64
event                 2550 non-null object
film_date             2550 non-null int64
languages             2550 non-null int64
main_speaker          2550 non-null object
name                  2550 non-null object
num_speaker           2550 non-null int64
published_date        2550 non-null int64
ratings               2550 non-null object
related_talks         2550 non-null object
speaker_occupation    2544 non-null object
tags                  2550 non-null object
title                 2550 non-null object
url                   2550 non-null object
views                 2550 non-null int64
dtypes: int64(7), object(10)
memory usage: 338.8+ KB


,comments,duration,film_date,languages,num_speaker,published_date,views
count,2550.000000,2550.000000,2.550000e+03,2550.000000,2550.000000,2.550000e+03,2.550000e+03
mean,191.562353,826.510196,1.321928e+09,27.326275,1.028235,1.343525e+09,1.698297e+06
std,282.315223,374.009138,1.197391e+08,9.563452,0.207705,9.464009e+07,2.498479e+06
min,2.000000,135.000000,7.464960e+07,0.000000,1.000000,1.151367e+09,5.044300e+04
25%,63.000000,577.000000,1.257466e+09,23.000000,1.000000,1.268463e+09,7.557928e+05
50%,118.000000,848.000000,1.333238e+09,28.000000,1.000000,1.340935e+09,1.124524e+06
75%,221.750000,1046.750000,1.412964e+09,33.000000,1.000000,1.423432e+09,1.700760e+06
max,6404.000000,5256.000000,1.503792e+09,72.000000,5.000000,1.506092e+09,4.722711e+07


### <font color = "blue"> Self-Assessment: Pandas </font>

How many talks are in the TED Talks data frame?

In [4]:
df.shape[0]

2550

### <font color = "blue"> Self-Assessment: Prerequisites </font> 

Select TED talks with these prerequisites:

1. talks with duration of at least 5 minutes (i.e. 300 seconds)
2. talks with only 1 speaker
3. talks in the top 90\% of views (exclude the bottom 10\%)

Also inspect the number of talks that made the cut.

In [5]:
df_filtered = df[(df['duration'] >= 300) & (df['num_speaker'] == 1) & (df['views'] >= df['views'].quantile(.9))]
df_filtered
df_filtered.shape[0]

232

### <font color = "blue"> Self-Assessment: Compute a Metric, Sort and Print </font>  

In the absence of numerical ratings here, use the ratio of the number of comments per 1000 views as a metric to sort the TED talks and print the 10 with the highest ratios.  

Display only the description, the main speaker, and the number of views.

In [6]:
views_blocks = df['views']/1000
df['new_rating'] = round(df['comments']/views_blocks,2)
df_sorted = df.sort_values('new_rating', ascending=False)
df_sorted[['description','main_speaker','views','new_rating']]

,description,main_speaker,views,new_rating
744,Hours before New York lawmakers rejected a key...,Diane J. Savino,292395,2.22
803,David Bismark demos a new system for voting th...,David Bismark,543551,1.53
96,Richard Dawkins urges all atheists to openly s...,Richard Dawkins,4374792,1.46
694,Filmmaker Sharmeen Obaid-Chinoy takes on a ter...,Sharmeen Obaid-Chinoy,1057238,1.42
954,Janet Echelman found her true voice as an arti...,Janet Echelman,1832930,1.36
...,...,...,...,...
2273,It happens to all of us: you unsubscribe from ...,James Veitch,5711833,0.01
2446,"Carolyn Jones spent five years interviewing, p...",Carolyn Jones,1079349,0.01
2475,"Meet Sharon Terry, a former college chaplain a...",Sharon Terry,1015849,0.01
2534,What the astronauts felt when they saw Earth f...,Benjamin Grant,646174,0.00


## Knowledge-Based Recommender

For this example we will use the TED Talks data set that you have already loaded to build a knowledge-based recommender by soliciting the desired publication year and word rating from the user.

### <font color = "blue"> Self-Assessment: Dealing with Dates </font>

Extract the year of the talk from the feature called **published_date** and put it in a new variable called **published_year**.  

First, the film dates need to be converted to datetime objects and then extract the year of the film date.  However, for the TED Talks data, include the argument *unit='s'* in the **to_datetime()** function in order to convert the dates correctly (based on the number of seconds to the unix epoch start).

Then convert **published_year** to an integer data type and be sure that there are no NAT values among them.

In [7]:
df['published_year'] = pd.to_datetime(df['published_date'], unit="s").dt.year.astype('int32')
df

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views,new_rating,published_year
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,0.10,2006
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,0.08,2006
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292,0.08,2006
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,0.12,2006
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,0.05,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2545,17,"Between 2008 and 2016, the United States depor...",476,TED2017,1496707200,4,Duarte Geraldino,Duarte Geraldino: What we're missing in the de...,1,1505851216,"[{'id': 3, 'name': 'Courageous', 'count': 24},...","[{'id': 2596, 'hero': 'https://pe.tedcdn.com/i...",Journalist,"['TED Residency', 'United States', 'community'...",What we're missing in the debate about immigra...,https://www.ted.com/talks/duarte_geraldino_wha...,450430,0.04,2017
2546,6,How can you study Mars without a spaceship? He...,290,TED2017,1492992000,3,Armando Azua-Bustos,Armando Azua-Bustos: The most Martian place on...,1,1505919737,"[{'id': 22, 'name': 'Fascinating', 'count': 32...","[{'id': 2491, 'hero': 'https://pe.tedcdn.com/i...",Astrobiologist,"['Mars', 'South America', 'TED Fellows', 'astr...",The most Martian place on Earth,https://www.ted.com/talks/armando_azua_bustos_...,417470,0.01,2017
2547,10,Science fiction visions of the future show us ...,651,TED2017,1492992000,1,Radhika Nagpal,Radhika Nagpal: What intelligent machines can ...,1,1506006095,"[{'id': 1, 'name': 'Beautiful', 'count': 14}, ...","[{'id': 2346, 'hero': 'https://pe.tedcdn.com/i...",Robotics engineer,"['AI', 'ants', 'fish', 'future', 'innovation',...",What intelligent machines can learn from a sch...,https://www.ted.com/talks/radhika_nagpal_what_...,375647,0.03,2017
2548,32,In an unmissable talk about race and politics ...,1100,TEDxMileHigh,1499472000,1,Theo E.J. Wilson,Theo E.J. Wilson: A black man goes undercover ...,1,1506024042,"[{'id': 11, 'name': 'Longwinded', 'count': 3},...","[{'id': 2512, 'hero': 'https://pe.tedcdn.com/i...",Public intellect

### <font color = "blue"> Self-Assessment: Stringified Dictionaries </font>

Since we will be asking the user to enter a descriptive word rating to select a talk and the feature  **ratings** is a stringified dictionary, convert the list of dictionaries into a list of strings and explode the ratings column in a separate data frame.  Call the new data column **word_ratings**.

In [8]:
list_of_dicts = []
list_of_lists = []

for i in range(df.shape[0]):
    i_dicts = eval(df['ratings'].iloc[i])
    list_of_dicts.extend(i_dicts)

    new_val = []
    for dc in i_dicts:
        new_val.append(dc['name'])
        
    list_of_lists.append(new_val)

df['word_ratings'] = list_of_lists
separate_df = pd.DataFrame.from_dict(list_of_dicts, orient='columns')

### <font color = "blue"> Self-Assessment: Create the Knowledge-Based Recommender </font>

1. Print a list of the descriptive word ratings for the user to choose from. (*Hint:  you can do this with the line* print(gen_ted['word_ratings'].unique()))


2. Ask the user to enter answers to the following questions:

    - Enter a descriptive word for rating.
    - Enter the earliest year published for the talk (between 2006 and 2017).
    - Enter the latest year published for the talk (between 2006 and 2017).


3. Display the top 5 recommended talks according to the "comments per 1000 views" ratio based on the user preferences given.  


4. Display only the main speaker, the name of the talk, the year published, and the comments per thousand views ratio.


5.  Show the results for the word rating "obnoxious" and published years between 2009 and 2014.

In [9]:
separate_df['name'].unique()
desc_string = input("Enter a descriptive word for rating")
earliest_year = input("Enter the earliest year published for the talk between 2006 and 2017")
latest_year = input("Enter the latest year published for the talk between 2006 and 2017")

Enter a descriptive word for ratingPersuasive
Enter the earliest year published for the talk between 2006 and 20172009
Enter the latest year published for the talk between 2006 and 20172011


In [10]:
filtered_indices = separate_df[(separate_df['name'] == desc_string)].index
df2 = df.filter(items=filtered_indices, axis=0)
df2 = df2[(df2['published_year'] >= int(earliest_year)) & (df2['published_year'] <= int(latest_year))]
df2

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views,new_rating,published_year,word_ratings
360,116,Biochemist Kary Mullis talks about the basis o...,1772,TED2002,1012608000,19,Kary Mullis,Kary Mullis: Play! Experiment! Discover!,1,1231117200,"[{'id': 10, 'name': 'Inspiring', 'count': 89},...","[{'id': 237, 'hero': 'https://pe.tedcdn.com/im...",Biochemist,"['DNA', 'biology', 'curiosity', 'history', 'mi...",Play! Experiment! Discover!,https://www.ted.com/talks/kary_mullis_on_what_...,532759,0.22,2009,"[Inspiring, Funny, Fascinating, Informative, P..."
375,42,"Asteroid strikes get all the coverage, but ""Me...",1181,TED2008,1204070400,22,Peter Ward,Peter Ward: A theory of Earth's mass extinctions,1,1233018000,"[{'id': 22, 'name': 'Fascinating', 'count': 17...","[{'id': 499, 'hero': 'https://pe.tedcdn.com/im...",Paleontologist,"['dinosaurs', 'disease', 'environment', 'histo...",A theory of Earth's mass extinctions,https://www.ted.com/talks/peter_ward_on_mass_e...,837508,0.05,2009,"[Fascinating, Informative, Jaw-dropping, Confu..."
389,201,The SETI Institute's Jill Tarter makes her TED...,1283,TED2009,1233792000,26,Jill Tarter,Jill Tarter: Join the SETI search,1,1235091600,"[{'id': 11, 'name': 'Longwinded', 'count': 87}...","[{'id': 306, 'hero': 'https://pe.tedcdn.com/im...",Astronomer,"['TED Prize', 'exploration', 'extraterrestrial...",Join the SETI search,https://www.ted.com/talks/jill_tarter_s_call_t...,1263442,0.16,2009,"[Longwinded, Unconvincing, Persuasive, Fascina..."
402,223,"Athlete, actor and activist Aimee Mullins talk...",598,TED2009,1233792000,42,Aimee Mullins,Aimee Mullins: My 12 pairs of legs,1,1236733200,"[{'id': 9, 'name': 'Ingenious', 'count': 199},...","[{'id': 443, 'hero': 'https://pe.tedcdn.com/im...",Athlete and actor,"['activism', 'design', 'fashion', 'future', 'p...",My 12 pairs of legs,https://www.ted.com/talks/aimee_mullins_prosth...,3472415,0.06,2009,"[Ingenious, Fascinating, Persuasive, Inspiring..."
416,109,Jacek Utko is an extraordinary Polish newspape...,364,TED2009,1233792000,31,Jacek Utko,Jacek Utko: Can design save newspapers?,1,1238461200,"[{'id': 10, 'name': 'Inspiring', 'count': 534}...","[{'id': 75, 'hero': 'https://pe.tedcdn.com/ima...",Newspaper designer,"['business', 'creativity', 'culture', 'design'...",Can design save newspapers?,https://www.ted.com/talks/jacek_utko_asks_can_...,1055886,0.10,2009,"[Inspiring, Jaw-dropping, Ingenious, Unconvinc..."
430,113,Margaret Wertheim leads a project to re-create...,933,TED2009,1233878400,22,Margaret Wertheim,Margaret Wertheim: The beautiful math of coral,1,1240189200,"[{'id': 22, 'name': 'Fascinating', 'count': 44...","[{'id': 198, 'hero': 'https://pe.tedcdn.com/im...",Figurer,"['art', 'design', 'math', 'oceans', 'science',...",The beautiful math of coral,https://www.ted.com/talks/margaret_wertheim_cr...,1280829,0.09,2009,"[Fascinating, Beautiful, Informative, Jaw-drop..."
444,277,Louise Fresco shows us why we should celebrate...,1080,TED2009,1233792000,22,Louise Fresco,Louise Fresco: We need to feed the whole world,1,1241658000,"[{'id': 8, 'name': 'Informative', 'count': 212...","[{'id': 263, 'hero': 'https://pe.tedcdn.com/im...",Food and agriculture expert,"['activism', 'agriculture', 'business', 'consu...",We need to feed the whole world,https://www.ted.com/talks/louise_fresco_on_fee...,922158,0.30,2009,"[Informative, Courageous, Longwinded, Fascinat..."
458,103,"In this brief talk from TED U 2009, Jonathan D...",394,TED2009,1233878400,29,Jonathan Drori,Jonathan Drori: Why we're storing billions of ...,1,1243472400,"[{'id': 22, 'name': 'Fascinating', 'count': 14...","[{'id': 476, 'hero': 'https://pe.tedcdn.com/im...",Educator,"['biodiversity', 'biology', 'ecology', 'enviro...",Why we're storing billions of seeds,https://www.ted.com/talks/jonathan_drori_why_w...,555846,0.19,2009,"[Fascinating, Informative, Ingenious, Unconvin..."
470,103,

## Content-Based Recommender

For this example we will use the TED Talks data set that you have already loaded to build a content-based recommender based on the descriptions of the talks.  This will correspond to the **plot description-based recommender**.

### <font color = "blue"> Self-Assessment: TF-IDF Vectors </font>

From the original TED Talks data frame that use in this lesson, create the TF-IDF (term frequency - inverse document frequency) matrix from the descriptions of the talks.  The TF-IDF is high where a rare term is present or frequent in a document and TF-IDF is near zero where a term is absent from a document, or abundant across all documents.

The feature name in the data frame is **description**.

Output the shape of the TF-IDF matrix you create. The number of rows corresponds to the number of TED talks in the data frame and the number of columns represents the number of unique terms. 

In [11]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stopwords
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df['description'] = df['description'].fillna('')

#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(df['description'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(2550, 15162)

### <font color = "blue"> Self-Assessment: Create the Content-Based Recommender Based on Cosine Similarity </font>

Compute the cosine similarity score for all of the TED talks in the data frame. Next build the recommender to request the name of a TED talk in the data frame and provide the top 5 recommended talks based on the similarity of the descriptions with the name of the talk supplied.

Show that it works by getting the top 5 recommended talks that are similar to the talk named "Tyler Cowen: Be suspicious of simple stories".

In [13]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [15]:
#Construct a reverse mapping of indices and movie titles, and drop duplicate titles, if any
indices = pd.Series(df.index, index=df['name']).drop_duplicates()

In [20]:
# Function that takes in movie title as input and gives recommendations 
def content_recommender(title, cosine_sim=cosine_sim, df=df, indices=indices):
    # Obtain the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies. Ignore the first movie.
    sim_scores = sim_scores[1:6]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['name'].iloc[movie_indices]

In [21]:
#Get recommendations for The Lion King
content_recommender('Tyler Cowen: Be suspicious of simple stories')

1910    Tom Wujec: Got a wicked problem? First, tell m...
451     Dan Ariely: Are we in control of our own decis...
1115         Mikko Hypponen: Three types of online attack
1405           Ronny Edry: Israel and Iran: A love story?
2361     Sisonke Msimang: If a story moves you, act on it
Name: name, dtype: object